<a href="https://colab.research.google.com/github/balszeg/Thesis/blob/main/YOLOv4_KITTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Building DarkNet

Important to set the Colab runtime type to **GPU**.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet
!make

In [ ]:
# mount Google Drive + creat symbolic link to short its path
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

##Dataset preprocessing

In [ ]:
# install TensorFlow datasets, TensorFlow, and Matplotlib
!pip install -q tensorflow-datasets tensorflow matplotlib

In [ ]:
# import them
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
# download KITTI
# interrupt the process after the downloading, before .tfrecord creation start
ds = tfds.load('kitti', split=['train','validation','test'] , shuffle_files=False, data_dir='/content/darknet/data/kitti')
assert isinstance(ds, tf.data.Dataset)

In [ ]:
%cd /content/darknet/data/kitti/downloads/

In [ ]:
!unzip s3.eu-central-1_avg-kitti_object_label_2okdQhuxqbGezqixznEkoPaq2KIsnILcOA-iVL6sLd5o.zip
!unzip s3.eu-central-1_avg-kitti_object_image_2wcWQdlsC8TqO5-og-I_tnbmEYGA1YQAoPOGOASRGmis.zip

In [ ]:
# delete the zipped files to free storage space up
%cd /content/darknet/data/kitti/downloads
!find . -name "*.zip" -type f -delete

In [ ]:
# clone the YOLO converter repository
!git clone https://github.com/ssaru/convert2Yolo

In [ ]:
# install the requirements
%cd /content/darknet/data/kitti/downloads/convert2Yolo
!pip3 install -r requirements.txt

In [ ]:
# convert the labels to YOLO form
!mkdir /content/darknet/data/kitti/downloads/converted
!python3 example.py --datasets KITTI --img_path /content/darknet/data/kitti/downloads/training/image_2 --label /content/darknet/data/kitti/downloads/training/label_2 --convert_output_path /content/darknet/data/kitti/downloads/converted --img_type ".png" --manifest_path ./ --cls_list_file /content/gdrive/MyDrive/yolov4/obj.names

## Configuration

In [ ]:
# get the necessary config file from drive
# the parameter selection detailed in the thesis
%cd /content/darknet/cfg
!cp /mydrive/yolov4/yolov4-obj.cfg /content/darknet/cfg
!ls /content/darknet/cfg/

In [ ]:
# get the necessary .names and .data files from drive
!cp /mydrive/yolov4/obj.names /content/darknet/data
!cp /mydrive/yolov4/obj.data  /content/darknet/data

In [ ]:
# get the data path generator Python scripts from drive
!cp /mydrive/yolov4/generate_train.py /content/darknet
!cp /mydrive/yolov4/generate_test.py /content/darknet

In [ ]:
# excute them
%cd /content/darknet
!python generate_train.py
!python generate_val.py

In [ ]:
# copy the generated .txt inside the DarkNet folder
!cp /content/darknet/data/kitti/downloads/testing/val.txt /content/darknet/data
!cp /content/darknet/data/kitti/downloads/training/train.txt /content/darknet/data

In [ ]:
# copy the labels .txt to the same folders as the images
# the YOLO demands this, so it could access to data
!cp -a "/content/darknet/data/kitti/downloads/training/label_2/." "/content/darknet/data/kitti/downloads/training/image_2/"
!cp -a "/content/darknet/data/kitti/downloads/validation/label_2/." "/content/darknet/data/kitti/downloads/validation/image_2/"
!ls '/content/darknet/data/kitti/downloads/training/image_2/'

## Training

Pressing **Ctrl+Shift+I** (or **F12**) the browser console opens up. Copying this script is prevent the Colab's timeout.
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

In [ ]:
# training from scratch
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg -dont_show -map

In [ ]:
# training with pretrained weights
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show -map

## Evaluate

In [ ]:
# calculate metrics
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights

In [ ]:
# testing on video
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg yolov4-obj_last.weights -dont_show /mydrive/video/traffic_video.mp4 -i 0 -out_filename /mydrive/video/results.avi